In [19]:
## Dynamic Light Scattering ##

## Data Analysis Process - Matlab Code converter (Version 1.1) ##

## D.Sapalidis, St. Gallen, Switzerland, 28.06.2024 ##


import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import os
import pathlib

# Measurement numbers
j_90 = 73
j_173 = 76

# Ονόματα αρχείων μετρήσεων
file_90 = f'20190603_cubA4_90_56{j_90}.txt'
file_173 = f'20190603_cubA4_173_56{j_173}.txt'

# Example usage of the function
input_directory = r"C:\Users\Dimitris\Desktop\DLLLLLS\Results"
output_directory = r"C:\Users\Dimitris\Desktop\DLLLLLS\Graphs"

# Convert Windows paths to POSIX paths for compatibility
input_directory = pathlib.PureWindowsPath(input_directory).as_posix()
output_directory = pathlib.PureWindowsPath(output_directory).as_posix()


# Print converted directories for verification
print("Input directory (POSIX format):", input_directory)
print("Output directory (POSIX format):", output_directory)


# Discard bad results
Discard_list = []  # Sequential order of measurements to discard for global fitting
Discard_list_total = []  # Sequential order of measurements not to show

Temperature = 25  # Example, user-defined value
Temp_Kelvin = Temperature + 273.15
Boltzmann = 1.38065e-23  # J.K-1, example value, user-defined
lambda_val = 0.6328e-9  # Example, user-defined
rIndex = 1.37  # Example, user-defined
visc = 0.887e-3  # Example, user-defined

# Options
showPartSize = 1
showR2 = 1
mergeFigs = 1
showHor = 1
showLegd = 2
PrintFigs = 1
NormLz = 1

# Initial guesses for fitting parameters
x0_1 = np.array([0, 1, 200e-9, 2e-4])
lb_1 = np.array([0, 0, 10e-9, 0])
up_1 = np.array([2, 2, 5000e-9, 20])


# Second fit (linear)
x1_1 = 2e-14
x1_2 = 2e-14

# Miscellaneous
XPicSize1 = 3
YPicSize1 = 4
XPicSize2 = 3
YPicSize2 = 4

# Print results function
def print_results(FitResultsTable, mean_PDI_90, mean_PDI_173, D1, Rh1, Dh1, SE_Dh1):
    print("Fit Results Table:")
    print(FitResultsTable)
    print("Global Fit:")
    print(" ' '   'diffusion / m2/s'   'Rh / nm'   'Dh / nm'   'SE Dh / nm'   'PDI 90Ί'   'PDI 173Ί'   'PDI'")
    print(f" 'fast mode'   {D1}   {Rh1*10**9}   {Dh1*10**9}   {SE_Dh1*10**9}   {mean_PDI_90}   {mean_PDI_173}")

def load_measurement_data(file_pattern, j, directory):
    measurements = []
    for i in range(j, j + 2):
        filename = os.path.join(directory, file_pattern.format(i))
        try:
            data = pd.read_csv(filename, delimiter='\t', header=None, names=['Column1', 'Column2', 'Column3'], encoding='latin1')
            measurements.append(data)
        except UnicodeDecodeError as e:
            print(f"UnicodeDecodeError: Failed to decode {filename} with 'latin1' encoding.")
            print(e)
        except Exception as e:
            print(f"Error reading file {filename}: {e}")
    return measurements


# Example usage:
# Load measurement data
measurements_90 = load_measurement_data(file_90, j_90, input_directory)
measurements_173 = load_measurement_data(file_173, j_173, input_directory)

 
idx = 1
j1 = j_90
data90 = {}

while idx <= 5 and j_90 <= j1 + 4:
    try:
        with open(f'{file_90}{j_90}') as f:
            fileRead = f.readlines()

        start_line1 = next(i for i, line in enumerate(fileRead) if 'Correlation' in line)

        data90[idx] = np.loadtxt(f'{file_90}{j_90}', delimiter=',', skiprows=start_line1)

        # Getting additional info if required (similar logic as MATLAB)
        if showHor == 1:
            info_lines = [12, 16, 37, 38]
            for i_info in info_lines:
                with open(f'{file_90}{j_90}') as file_info:
                    line_file_info = None
                    for _ in range(i_info):
                        line_file_info = file_info.readline()

                # Extracting specific information based on line type
                if i_info == info_lines[0]:
                    Trans90[idx] = float(line_file_info.split(',')[1].strip())
                elif i_info == info_lines[1]:
                    Counts90[idx] = float(line_file_info.split(',')[1].strip())
                elif i_info == info_lines[2]:
                    Hor_Zav90[idx] = float(line_file_info.split(',')[1].strip())
                elif i_info == info_lines[3]:
                    Hor_PDI90[idx] = float(line_file_info.split(',')[1].strip())

    except Exception as e:
        print(f'Error processing file {file_90}{j_90}: {e}')

    idx += 1
    j_90 += 1

idx = 1
j1 = j_173
data173 = {}

while idx <= 5 and j_173 <= j1 + 4:
    try:
        with open(f'{file_173}{j_173}') as f:
            fileRead = f.readlines()

        start_line1 = next(i for i, line in enumerate(fileRead) if 'Correlation' in line)

        data173[idx] = np.loadtxt(f'{file_173}{j_173}', delimiter=',', skiprows=start_line1)

        # Getting additional info if required (similar logic as MATLAB)
        if showHor == 1:
            info_lines = [12, 16, 37, 38]
            for i_info in info_lines:
                with open(f'{file_173}{j_173}') as file_info:
                    line_file_info = None
                    for _ in range(i_info):
                        line_file_info = file_info.readline()

                # Extracting specific information based on line type
                if i_info == info_lines[0]:
                    Trans173[idx] = float(line_file_info.split(',')[1].strip())
                elif i_info == info_lines[1]:
                    Counts173[idx] = float(line_file_info.split(',')[1].strip())
                elif i_info == info_lines[2]:
                    Hor_Zav173[idx] = float(line_file_info.split(',')[1].strip())
                elif i_info == info_lines[3]:
                    Hor_PDI173[idx] = float(line_file_info.split(',')[1].strip())

    except Exception as e:
        print(f'Error processing file {file_173}{j_173}: {e}')

    idx += 1
    j_173 += 1



# Initialize lists or dictionaries to store results
lag90 = {}
corr90a = {}
F90 = {}
X90 = {}
Res90 = {}

# Initialize lists or dictionaries to store results
lag173 = {}
corr173a = {}
F173= {}
X173 = {}
Res173 = {}

# Autocorrelation curves fitting at 90°
for idx in range(1, 6):
    try:
        lag90[idx] = data90[idx][:, 0] / 1e3  # Transform time units to milliseconds
        corr90a[idx] = data90[idx][:, 1]

        if NormLz == 1:
            for idx1 in range(len(corr90a)):
                corr90a[idx1] = corr90a[idx1] / np.mean(corr90a[idx1][:3])  # Normalize by mean of first 3 values

        xdata = lag90[idx]
        ydata = corr90a[idx]

        q_square_temp = (((4 * np.pi * rIndex / lambda_val) * np.sin((90 * np.pi / 180) / 2))**2)
        x0_g = x0_1.copy()
        lb_g = lb_1.copy()
        up_g = up_1.copy()
        x0_g[2] = 0.001 * q_square_temp * Boltzmann * Temp_Kelvin / (3 * np.pi * visc * x0_1[2])
        lb_g[2] = 0.001 * q_square_temp * Boltzmann * Temp_Kelvin / (3 * np.pi * visc * up_1[2])
        up_g[2] = 0.001 * q_square_temp * Boltzmann * Temp_Kelvin / (3 * np.pi * visc * lb_1[2])

        # Define cumulantDLS function
        def cumulantDLS(x, xdata):
            return x[0] + x[1] * (np.exp(-x[2] * xdata) * (1 + 0.5 * x[3] * xdata**2))**2

        # Perform curve fitting
        x, resnormS, residuals = lsqcurvefit(cumulantDLS, x0_g, xdata, ydata, lb_g, up_g)
        F90[idx] = cumulantDLS(x, xdata)
        X90[idx] = x
        Res90[idx] = residuals

    except Exception as e:
        print(f'Error fitting autocorrelation curve at 90° for idx={idx}: {e}')

# Autocorrelation curves fitting at 173°
for idx in range(1, 6):
    try:
        lag173[idx] = data173[idx][:, 0] / 1e3  # Transform time units to milliseconds
        corr173a[idx] = data173[idx][:, 1]

        if NormLz == 1:
            for idx1 in range(len(corr173a)):
                corr173a[idx1] = corr173a[idx1] / np.mean(corr173a[idx1][:3])  # Normalize by mean of first 3 values

        xdata = lag173[idx]
        ydata = corr173a[idx]

        q_square_temp = (((4 * np.pi * rIndex / lambda_val) * np.sin((173 * np.pi / 180) / 2))**2)
        x0_g = x0_1.copy()
        lb_g = lb_1.copy()
        up_g = up_1.copy()
        x0_g[2] = 0.001 * q_square_temp * Boltzmann * Temp_Kelvin / (3 * np.pi * visc * x0_1[2])
        lb_g[2] = 0.001 * q_square_temp * Boltzmann * Temp_Kelvin / (3 * np.pi * visc * up_1[2])
        up_g[2] = 0.001 * q_square_temp * Boltzmann * Temp_Kelvin / (3 * np.pi * visc * lb_1[2])

        # Define cumulantDLS function
        def cumulantDLS(x, xdata):
            return x[0] + x[1] * (np.exp(-x[2] * xdata) * (1 + 0.5 * x[3] * xdata**2))**2

        # Perform curve fitting
        x, resnormS, residuals = lsqcurvefit(cumulantDLS, x0_g, xdata, ydata, lb_g, up_g)
        F173[idx] = cumulantDLS(x, xdata)
        X173[idx] = x
        Res173[idx] = residuals

    except Exception as e:
        print(f'Error fitting autocorrelation curve at 173° for idx={idx}: {e}')

import numpy as np



# Assuming X90 and X173 are already defined with results from cumulantDLS fitting
# Initialize lists or dictionaries to store results
angle = np.zeros(len(X90) + len(X173))
G1 = np.zeros(len(X90) + len(X173))
Baseline = np.zeros(len(X90) + len(X173))
Amplitude = np.zeros(len(X90) + len(X173))
variance = np.zeros(len(X90) + len(X173))

# Get angles automatically
for angle_idx_1 in range(len(X90)):
    angle[angle_idx_1] = 90

for angle_idx_2 in range(len(X173)):
    angle[len(X90) + angle_idx_2] = 173

angle_deg = angle
angle = angle * np.pi / 180
q = (4 * np.pi * rIndex / lambda_val) * np.sin(angle / 2)
q_square = q**2

# Get G1 and other parameters automatically
for angle_idx_1 in range(len(X90)):
    G1[angle_idx_1] = X90[angle_idx_1][2]
    Baseline[angle_idx_1] = X90[angle_idx_1][0]
    Amplitude[angle_idx_1] = X90[angle_idx_1][1]
    variance[angle_idx_1] = X90[angle_idx_1][3]

for angle_idx_2 in range(len(X173)):
    G1[len(X90) + angle_idx_2] = X173[angle_idx_2][2]
    Baseline[len(X90) + angle_idx_2] = X173[angle_idx_2][0]
    Amplitude[len(X90) + angle_idx_2] = X173[angle_idx_2][1]
    variance[len(X90) + angle_idx_2] = X173[angle_idx_2][3]

# Now you can use angle, G1, Baseline, Amplitude, variance as needed

# Initialize dictionaries to store additional results
G2 = {}

# Calculate G2 and other parameters automatically
for angle_idx_1 in range(len(X90)):
    # Assuming you have already defined the necessary parameters like q_square, Boltzmann, Temp_Kelvin, visc
    G2[angle_idx_1] = 0.001 * q_square[angle_idx_1] * Boltzmann * Temp_Kelvin / (3 * np.pi * visc * G1[angle_idx_1])

for angle_idx_2 in range(len(X173)):
    G2[len(X90) + angle_idx_2] = 0.001 * q_square[len(X90) + angle_idx_2] * Boltzmann * Temp_Kelvin / (3 * np.pi * visc * G1[len(X90) + angle_idx_2])

# FIT diffusion coefficient
xdata1 = q_square
ydata1 = G1

xdata1[Discard_list] = []
ydata1[Discard_list] = []

x0 = 2e-14
x, resnormS1 = lsqcurvefit(linear0, x0, xdata1, ydata1)
fit1 = linear0(x, xdata1)

D1 = x * 1000  # Convert to m^2/s
Rh1 = (Boltzmann * Temp_Kelvin) / (6 * np.pi * visc * D1)
Dh1 = Rh1 * 2

R2_1 = 1 - resnormS1 / np.sum(ydata1**2)
SE_D1 = 1000 * np.sqrt(resnormS1 / (len(ydata1) - 1)) / np.sqrt(np.sum(xdata1**2))
SE_Rh1 = (SE_D1 / D1) * Rh1
SE_Dh1 = (SE_D1 / D1) * Dh1

# Calculate individual D and Dh
D1_indiv = 1000 * G1 / q_square  # m^2 / s
D2_indiv = 1000 * G2 / q_square  # m^2 / s

Dh1_indiv = 10**9 * (Boltzmann * Temp_Kelvin) / (3 * np.pi * visc * D1_indiv)  # m
Dh2_indiv = 10**9 * (Boltzmann * Temp_Kelvin) / (3 * np.pi * visc * D2_indiv)  # m

if np.sum(G2 == 0) != 0:
    D3_indiv = 1000 * G1[G2 == 0] / q_square[G2 == 0]  # m^2 / s
    Dh3_indiv = 10**9 * (Boltzmann * Temp_Kelvin) / (3 * np.pi * visc * D3_indiv)  # m



# Calculate PDI table
PDI_table = variance / (G1**2)
PDI_valid = PDI_table.copy()
PDI_valid = np.delete(PDI_valid, Discard_list)
mean_PDI = np.mean(PDI_valid[PDI_valid != 0])

# Prepare PDI for 90° and 173° angles
PDI_90 = PDI_table[:len(X90)]
PDI_90_valid = PDI_90.copy()
PDI_90_valid = np.delete(PDI_90_valid, Discard_list_1)
mean_PDI_90 = np.mean(PDI_90_valid[PDI_90_valid != 0])

PDI_173 = PDI_table[len(X90):len(X173) + len(X90)]
PDI_173_valid = PDI_173.copy()
PDI_173_valid = np.delete(PDI_173_valid, Discard_list_2 - len(X90))
mean_PDI_173 = np.mean(PDI_173_valid[PDI_173_valid != 0])

std_PDI_valid = 0.5 * np.sqrt((np.std(PDI_173_valid))**2 + (np.std(PDI_90_valid))**2)
std_PDI_90_valid = np.std(PDI_90_valid)
std_PDI_173_valid = np.std(PDI_173_valid)

# Prepare FitResultsTable
FitResultsTable = np.column_stack((angle_deg, Amplitude, G1, Dh1_indiv, variance, PDI_table, Baseline))
print(' Fitting Results using the Cumulants analysis ')
print('        angle      amplitude    Gamma1 /ms-1 Dh_ind /m   variance    PDI        baseline ')

# Remove rows specified by Discard_list_total
FitResultsTable = np.delete(FitResultsTable, Discard_list_total, axis=0)

# Print the final FitResultsTable
print(FitResultsTable)

# Define the color map (cmap1 and cmap2) as RGB values
cmap1 = np.array([[0, 0, 1], [0, 0.75, 0], [1, 0, 0], [0.5, 0, 1], [1, 0.5, 0], [0, 0, 0], [0.5, 0.5, 0.5], [0.5, 0.25, 0.1]])
cmap2 = cmap1

# Plotting Figure 1
fig1, ax1 = plt.subplots()
plot_list_90 = [i for i in range(len(X90)) if i not in Discard_list_total_1]
plot_list_173 = [i for i in range(len(X173)) if i not in Discard_list_total_2 - len(X90)]

for idx in plot_list_90:
    if idx in Discard_list:
        ax1.semilogx(lag90[idx], corr90a[idx], '-', color=[0.5, 0.5, 0.5], linewidth=1.5)
    else:
        ax1.semilogx(lag90[idx], F90[idx], '-', color=cmap1[idx], linewidth=1.5)
        ax1.semilogx(lag90[idx], corr90a[idx], 'o', markersize=5, markeredgewidth=1, markeredgecolor='black', markerfacecolor=cmap1[idx])

for idx in plot_list_173:
    if idx + len(plot_list_90) in Discard_list:
        ax1.semilogx(lag173[idx], F173[idx], '-', color=[0.5, 0.5, 0.5], linewidth=1.5)
    else:
        ax1.semilogx(lag173[idx], F173[idx], '-', color=cmap2[idx], linewidth=1.5)
        ax1.semilogx(lag173[idx], corr173a[idx], 's', markersize=5, markeredgewidth=1, markeredgecolor='black', markerfacecolor=cmap1[idx])

# Set labels and ticks
ax1.set_xlabel(r'$\tau$ / ms', fontsize=11)
ax1.set_ylabel(r'$G_2(\tau) - 1$', fontsize=11)
ax1.set_xlim([0.5e-3, 2e2])
ax1.set_xticks([1e-3, 1e-2, 1e-1, 1, 10, 100])
ax1.set_ylim([0, 1])
ax1.tick_params(axis='both', which='both', direction='inout', labelsize=10)

# Adding legend and text on figure
if showLegd == 1:
    ax1.legend(['circles: 90Ί', 'fit', 'B', 'fit', 'C', 'fit', 'D', 'fit', 'squares: 173Ί', 'fit', 'F', 'fit', 'G', 'fit', 'H', 'fit', 'I', 'fit', 'J', 'fit', 'K', 'fit', 'L', 'fit'])

if showPartSize == 1:
    ax1.text(0.8e-3, 0.21, f'D_H = {2 * Rh1 * 10**9:.0f} nm', fontsize=6, color='b', horizontalalignment='left')

# Saving Figure 1
if PrintFigs == 1:
    fig1.savefig('temp1.tif', dpi=600)

# Plotting Figure 2
fig2, ax2 = plt.subplots()

# Set the new default color order cmap2
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=cmap2)

# Plot data according to FitModel == 1
ax2.plot(q_square, G1, 'o', markersize=5, markeredgewidth=1, markeredgecolor=[0.5, 0.5, 0.5], markerfacecolor=[0.5, 0.5, 0.5])
ax2.plot(xdata, ydata, 'o', markersize=5, markeredgewidth=1, markeredgecolor=[0, 0, 1], markerfacecolor=[0, 0, 1])
ax2.plot([0, xdata], [0, fit1], '-', color=[0, 0.9, 0])

# Set labels and ticks for Figure 2
ax2.set_xlabel(r'$q^2 / m^{-2}$', fontsize=11)
ax2.set_ylabel(r'$\Gamma / ms^{-1}$', fontsize=11)
ax2.tick_params(axis='both', which='both', direction='inout', labelsize=10)

# Adding text on Figure 2
if showR2 == 1:
    ax2.text(0.5, ax2.get_ylim()[1] - 1.2, f'R^2 = {R2_1:.3f}; N = {len(xdata)}; D_h_1 = {Dh1 * 10**9:.1f} ± {SE_Dh1 * 10**9:.1f} nm', fontsize=5, color=[0, 0.9, 0], horizontalalignment='left')
    ax2.text(0.5, ax2.get_ylim()[1] - 1.7, f'Dif_1 = {D1:.3f} ± {SE_D1:.3f} m^2/s', fontsize=5, color=[0, 0.9, 0], horizontalalignment='left')

# Saving Figure 2
if PrintFigs == 1:
    fig2.savefig('temp2.tif', dpi=600)

# Merging figures if mergeFigs == 1 (not implemented in Python, you may need additional libraries for image processing)
# Save Figures
if PrintFigs == 1:
    fig1.savefig(output_directory + 'temp1.tif', dpi=600)
    fig2.savefig(output_directory + 'temp2.tif', dpi=600)

plt.show()

plt.show()

Input directory (POSIX format): C:/Users/Dimitris/Desktop/DLLLLLS/Results
Output directory (POSIX format): C:/Users/Dimitris/Desktop/DLLLLLS/Graphs
Error processing file 20190603_cubA4_90_5673.txt73: [Errno 2] No such file or directory: '20190603_cubA4_90_5673.txt73'
Error processing file 20190603_cubA4_90_5673.txt74: [Errno 2] No such file or directory: '20190603_cubA4_90_5673.txt74'
Error processing file 20190603_cubA4_90_5673.txt75: [Errno 2] No such file or directory: '20190603_cubA4_90_5673.txt75'
Error processing file 20190603_cubA4_90_5673.txt76: [Errno 2] No such file or directory: '20190603_cubA4_90_5673.txt76'
Error processing file 20190603_cubA4_90_5673.txt77: [Errno 2] No such file or directory: '20190603_cubA4_90_5673.txt77'
Error processing file 20190603_cubA4_173_5676.txt76: [Errno 2] No such file or directory: '20190603_cubA4_173_5676.txt76'
Error processing file 20190603_cubA4_173_5676.txt77: [Errno 2] No such file or directory: '20190603_cubA4_173_5676.txt77'
Error pr

NameError: name 'lsqcurvefit' is not defined